# what is finetunning:  taking these general purpose models  and turning into  specailized models.
# reduce hallucinatons
# customize the model to a specific use case

# pros: data nearly unlimited data fits  , learn new information ,  correct incorrect info ,  less cost afterwards it samlled model , use RAG too,
# cos : more high quality data,upfront compute cost,needs some technical knowledge.


# Benefits of finetunnig LLM: performance , stop hallucinations, reduce unwanted info, privacy , prevent leakage, no breaches,cost, lower cost per request, increased transparency,reliability , control uptime,lower latency,moderations , 



In [4]:
import os
from lamini import Lamini

In [ ]:
from llama import BasicModelRunner
no_finetunned=BasicModelRunner("meta-llama/Llama-2-7b-hf")
non_finetuned_output=non_finetunned("What is your name?")


In [ ]:
finetuned_model=BasicModelRunner("meta-llama/Llama-2-7b-chat-hf")
finetuned_output=finetuned_model("What is your name?")


In [ ]:
print(non_finetuned("[INST] tell me how to train my dog?"))

      

In [ ]:
# where finetuning fit in : pretraining -> finetuning ->  again finetuning can be done.
# finetuning usually refers to training further can also be self-supervised unlabeled data,can be labeld data you curated, much less data needed.
# finetuning for generative tasks is not well defined: update entire model , not just part of it, smae training objetive : next token prediction.


In [ ]:
# what is finetuning doing for us:
# behave change : learning to respond more consitently
# learning ro focus
# gain knowledge: increasing knowledge of new specific concepts,
# correcting old incorrect info.
# both : 




In [ ]:
# tasks to finetune
# just test-in, text-out:
    # Extraction : text in , less text out
        # reading 
        # keywords, topics, routing, agents
    #expansion:text in, more text out
        # writing 
        # chat , write emails
# tasks clarity is key indicator of success
# clarity means knowing what's bad and good and better


In [ ]:
# first time finetunning
# 1. identify task by prompt engi. a LLM
# 2. find task that you see on LLM doing ~ok at.
# 3.pick one task.
# 4. get ~1000 inputs and outputs for the task.
# 5.finetune a small LLM on this data.


In [14]:
import jsonlines 
import itertools
import pandas as pd
import pprint import pprint
import datasets
from datasets import load_dataset


In [ ]:
pretrained_dataset=load_dataset('EleutherAI/pile',split='train',streaming=True)


In [ ]:
n=5
top_n=itertools.islice(pretrained_dataset,n)
for i in top_n:
    print(i)
    

In [ ]:
# steps to prepare the data.
# @ collect instruction - response pairs
# tokenize : pad,truncate
# split into train/text
# tokenize the data: convert the words  into numbers

from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("")
text='Hi , I am Pawan'
encoded_text=tokenizer(text)['input_ids']
print(encoded_text)
decoded_text=tokenizer.decode(endoded_text)



In [16]:
list_texts=['hi', How are you?','I am good','yes']
# padding
tokenizer.pad_token=tokenizer.eos_token
encoded_texts_longest=tokenizer(list_texts,padding=True)
print(encoded_texts_longest)


In [18]:
# truncation
encoded_texts_truncation=tokenizer(list_texts,max_length=3,truncation=True)
print(encoded_text_truncation['input_ids'])


In [ ]:
# truncate and padding
encoded_texts_truncation=tokenizer(list_texts,max_length=3,truncation=True,padding=True)
print(encoded_text_truncation['input_ids'])

In [19]:
def tokenize_function(examples):
    if 'question' in examples and 'answer' in examples:
        text=examples['question'][0]+examples['answers'][0]
    elif 'input' in examples and 'output' in examples:
        text=examples['input']+examples['output'][0]
    else:
        text=examples['text'][0]

    tokenizer.pad_token=tokenizer.eos_token
    tokenized_input=tokenizer(text,return_tensor='np',padding=True)
    max_length=min(tokenized_inputs['input_ids'].shape[1],2048)
    tokenizer.truncation_side='left'
    tokenized_inputs=tokenizer(text,return_tensors='np',truncation=True,max_length=max_length)
    return tokenized_inputs

    

In [ ]:
finetunnig_dataset_loaded=datasets.load_dataset('json',data_files=filename)
tokenized_dataset=finetuning_dataset_loaded.map(tokenize_function,batched=True,batch_size=1,drop_last_batch=True)
print(tokenized_dataset)


In [ ]:
tokenized_dataset=tokenized_dataset.ad_column('labels',tokenized_dataset['input_ids'])
split_dataset=tokenized_dataset.train_test_split(text_sie=0.1,shuffle=True,seed=123)
print(split_dataset)


In [ ]:
# training Process

# add training data
# calculate loss
# backprop through model
# update weights
# st hyperparameters like: learning rate,learning rate scheduler , optimize hyperparameter

In [ ]:
# for epoch in range(num_epochs):
#     for batch in train_dataloades:
#         outputs=model(**batch)
#         loss=outputs.loss
#         loss.backward()
#         optimizer.step()


In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import torch
import transformers

from utilities import *
from transformers import AutoTokenizer,AutoModelForCasualLM,AutoModelForCasualLm
from llama import BasicModelRunner
from llama import BasicModelRunner

logger=logging.getLogger(__name__)
global_config=None



In [ ]:
dataset_name='lamini_docs.jsonl'
dataset_path=''
use_hf=False
dataset_path=''
use_hf=True

In [ ]:
model_name=''
training_config={
    'model':{
        'pretrained_name':model_name,
        'max_length':2048,
    },
    'dataset':{
        'use_hf':use_hf,
        'path':dataset_path
    },
    'verbose':True
}


In [ ]:
tokenizer=AutoTokenizer.from_pretrained("")
tokenizer.pad_token=tokenizer.eos_token
train_dataset,text_dataset=tokenize_and_split_data(training_config,tokenizer)




In [ ]:
model=AutoModelFoerCasualLM.from_pretrained(model_name)

device_count=torch.cuda.device_count
if device_count>0:
    logger.debug("select gpu device")
    device=torch.device("cude")
else:
    logger.debug("Select CPU device")
    device=torch.device("CPU")

base_model.to(device)


In [ ]:
def inference(text,model,tokenize,max_input_tokens=1000,max_output_token=100):
    input_ids=tokenizer.encode(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=max_input_tokens
    )

    device=model.device
    generated_token_with_prompt=model.generate(input_ids=input_ids.to(device),max_length=max_output_tokens)
    generated_text_with_prompt=tokenizer.batch_decode(generate_tokens_with_prompt,skip_special_token=True)
    generated_text_answer=generated_text_with_prompt[0][len(text):]
    return  generated_text_answer


max_steps=3
trained_model_name=f'{max_steps}'
output_dir=trained_model_name

training_args=TrainingArguments(
    learning_rate=1.0e-5,
    num_train_epochs=1,
    max_steps=max_steps,
    per_device_train_batch_size=1,
    output_dir=output_dir,
    overwrite_output_dir=False,
    disable_tqdm=False,
    eval_steps=120,
    save_steps=120,
    warmup_steps=1,
    per_device_eval_batch_size=1,
    evaluation_strategy='steps',
    logging_strategy='steps',
    logging_steps=1,
    optim='adafactor',
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    logging_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)

    
    

In [ ]:
model_flops=(
    base_model.floating_point_ops(
        {
            'input_ids':torch.zeros(
                (1,training_config['mode']['max_length'])
            )
        }
    )
    *training_args.gradient_accumulation_steps
)
print(base_model)
print(base_mode.get_memory_footprint()/1e9)
print(model_flops/1e9)


In [ ]:
trainer=Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

training_output=trainer.train()

save_dir=f'{output_dir}/final'
trainer.save_model(save_dir)

finetuned_slightly_model=AutoodelForCasualLM.from_pretrained(save_dir,local_files_only=True)


finetuned_slightly_model.to(device)

text_question=test_dataset[0]['question']
print(text_question)
print(inference(text_question,finetuned_slightly_modee,tokenizer))


In [ ]:
test_answer=test_dataset[0]['answer']
print(test_answer)

In [ ]:
finetuned_longer_model=AutoModelForCasualLM.from_pretrained('')
toeknizer=AutoTokenizer.from_pretrained('')
finetuned_longer_model.to(device)
print(inference(test_question,finetuned_longer_model,tokenizer))



In [ ]:
bigger_finetuned_model=BasicModelRunner(model_name_to_id['bigger_model_name'])
bigger_finetuned_output=bigger_finetuned_model(test_question)
print(bigger_finetuned_output)


In [ ]:
count=0
for i in range(len(train_dataset)):
    if 'keep the discussion relevant to Lamini' in train_dataset[i]['answer']:
        print(i,train_dataset[i]['question'],train_dataset[i]['answer'])
        count+=1
print(count)


In [ ]:
base_model=AutoModelForCasualLM.from_pretrained('')
base_tokenizer=AutoTokenizer.from_pretrained('')
print(inference('what do you think of mars',base_model,base_tokenizer))


print(inference('what do you think of mars',finetuned_longer_model,tokenizer))

print(inference('what do you think of mars',finetuned_longer_model,tokenizer))


In [ ]:
model=BasicModelRunner('')
model.load_data_from_jsonlines('')
model.train(is_public=True)


In [ ]:
out=model.evaluate()


In [ ]:
lofd=[]
for e in out['eval_results']:
    q=f'{e['input']}'
    at=f'{e['outputs'][0]['output']}'
    ab=f'{e['outputs'][0]1['output']}'
    di={'question':q,'trained_model':at,'base_mode':ab}
    lofd.append(di)

df=pd.DataFrame.from_dict(lofd)
style_df=df.style.set_properties(**{'text-align':'left'})
style_df=style_df.style.set_properties(**{'vertical-align':'text-top'})


    

In [ ]:
# evaluation and iteration
dataset=datasets.load_dataset('')
test_dataset=dataset['text']


In [ ]:
model_name=''
model=AutoModelForCasualLM.from_pretrained('')
toeknizer=AutoTokenizer.from_pretrained(model_name)

def is_exact_match(a,b):
    return a.strip()==b.strip()

model.eval()



In [ ]:
text_question=text_dataset[0]['question']
generated_answer=inference(text_question,model,tokenizer)
print(is_exact_match(generated<answer,answer))


In [ ]:
n=10
metrics={'exact_matches':[]}
predictions=[]
for i,item in tqdm(enumerated(test_dataset)):
    question=item['question']
    answer=item['answer']

    try:
        predicted_answer=inference(question,model,tokenizer)
    except:
        continue
    predictions.append([predicted_answer,answer])
    exact_match=is_excat_match(generated_answer,answer)
    metrics['exact_matches'].append(exact_match)

    if i>n and n!=-1:
        break

print(sum(metrics['exact_matches']))




In [ ]:
df=pd.DataFrame(predictions,columns=['predicted_answer','target_answer'])
print(df)


In [ ]:
#prectical approach
# 1. figure task
# 2.collect data related to the task's input/output
# 3.generate data if you don't have enough data
# 4.finetune a small model
# 5.vary the amount of data you give the model
# 6. evaluate your LLM to know what's going well vs not
# 7. Collect more data to improve
# 8. Increase task complexity
# peft=parameter - efficient finetunning
# lora=low rank adaption of LLMs:
    # fewer trainble parameters: for GPT 3, 100000x less
    # less gpu memory: for GPT3 , 3x less
    # slightly below accuracy to finetuning
    # same inference latency
# train new weights in some layers,freeze main weights
    # new weights,rank decomposition matrices of original weights change
    # at inference, merge with main weights
